## Import Required Libraries 

In [1]:
import json
import time
import os
import csv
import json
import numpy as np
import pandas as pd
import random
from itertools import combinations
import math
import pickle

## Load Signature Matrix

In [2]:
# Open file containing Signature Matrix
fr = open('./data/signature_matrix_10_hashes.csv', 'r')
reader = csv.reader(fr)

# Initial signature matrix array
sig_mat = np.empty([10, 2262292], dtype=int)

# Loop through each row of signature matrix file
t0 = time.time()
count = 0
for row in reader:
    
    # Convert string of signature row to integers
    row_2_int = []
    for ele in row:
        row_2_int.append(int(ele))
    sig_mat[count] = row_2_int
    
    # Display status
    count = count + 1
    if count%100 == 0:
        print('Row', count, '-', time.time()-t0, 'sec')
        
# Close file
fr.close()

The signature matrix is loaded here instead of calculated in place due to memory constraints.

## Convert Signature Matrix to LSH Matrix 

In [3]:
# Transpose signature matrix for easier processing
sig_mat = sig_mat.T

# Open file to write LSH values
fw = open('./data/LSH_matrix_1rows_10bands.csv', 'w', newline='')
writer = csv.writer(fw, delimiter=',')

# Set number of rows and bands
bands = 10
num_rows = 1

# Loop through each track in signature matrix
t0 = time.time()
for track in range(0, len(sig_mat)):
    
    # Loop through each band and hash rows in band
    LSH_per_track = []
    for band in range(0, bands):
        LSH_val = ''
        for i in range(num_rows*(band), num_rows*(band+1)):
            LSH_val = LSH_val + str(sig_mat[track][i])
        
        # Append hash value to list
        LSH_per_track.append(LSH_val)
    
    # Write hash values to file
    writer.writerow(LSH_per_track)
    
    # Display status
    if int(track)%1000000 == 0:
        print('Track', track, '-', time.time()-t0, 'sec')
        
# Close file
fw.close()

# Clear out memory
LSH_val = []
LSH_per_track = []

Track 0 - 0.0 sec
Track 1000000 - 16.04891014099121 sec
Track 2000000 - 32.29577994346619 sec


Values within bands are hashed together simply being concatenating the row values together is strings.

## Get Candidate Pairs

In [4]:
# Open file containing LSH values
fr = open('./data/LSH_matrix_1rows_10bands.csv', 'r')
reader = csv.reader(fr, delimiter=',')
bands = 10
num_rows = 1

# Initialize dictionary to store hash values and candidate pairs
hash_value = {}
can_pairs = {}
for band in range(0,bands):
    hash_value[band] = {}
    can_pairs[band] = {}
    
# Initialize  counter
count = [0]*bands
track_num = 0
t0 = time.time()

# Loop through each track in LSH file
for track in reader:
    
    # Loop through each LSH value
    for band in range(0, len(track)):
        
        # Check if LSH value already exists in dictionary
        ID = hash_value[band].get(track[band], '')
        
        # Add new value to dictionaries
        if len(str(ID)) == 0:
            hash_value[band][track[band]] = count[band]
            can_pairs[band][count[band]] = list([track_num])
            count[band] = count[band] + 1
        
        # Add value to existing dictionary entry
        else:
            idx = can_pairs[band].get(ID, '')
            idx.append(track_num)
            can_pairs[band][ID] = idx

    # Display Progress
    track_num = track_num + 1
    if track_num%100000==0:
        print('Track', track_num, '-', time.time()-t0, 'sec')


Track 100000 - 1.4738233089447021 sec
Track 200000 - 3.0178606510162354 sec
Track 300000 - 4.452858209609985 sec
Track 400000 - 5.955823659896851 sec
Track 500000 - 7.522373676300049 sec
Track 600000 - 8.762340307235718 sec
Track 700000 - 10.406372547149658 sec
Track 800000 - 11.653377532958984 sec
Track 900000 - 12.880340337753296 sec
Track 1000000 - 14.658340692520142 sec
Track 1100000 - 15.866341829299927 sec
Track 1200000 - 17.02937602996826 sec
Track 1300000 - 18.18537425994873 sec
Track 1400000 - 20.019372940063477 sec
Track 1500000 - 21.14843440055847 sec
Track 1600000 - 22.289466619491577 sec
Track 1700000 - 23.41043519973755 sec
Track 1800000 - 24.53381323814392 sec
Track 1900000 - 25.659815788269043 sec
Track 2000000 - 27.681851625442505 sec
Track 2100000 - 28.78685235977173 sec
Track 2200000 - 29.896848440170288 sec


This first identifies unique hash value for each band. Those hash values are mapped to integer values to cut down on memory to store the hash values. Tracks containing the same hash value for a band are grouped together in a dictionary with their corresponding integer hash value.

In [13]:
#Initialize dictionary to store candidate pairs
cp_dict = dict()

# Loop through each band of LSH matrix
t0 = time.time()
for band in range(0,bands):
    
    # Loop through each unique LSH value
    count = 0
    for key, value in can_pairs[band].items():
        
        # If LSH value has moe than one track, create candidate pair
        if len(value) > 1:
            
            # Create combinations of track in LSH value
            combs = list(combinations(value, 2))

            # Process pair to list in a sorted order
            for single_pair in combs:
                single_pair = list(single_pair)
                single_pair.sort()
                
                # Add candidate pair to dictionary so that duplicates are discarded
                cp_dict[str(single_pair[0]) + ' ' + str(single_pair[1])]=1
        
        # Display status
        count = count+1
        if (count%100000 == 0) & (count > 0):
            print('Band:', band, 'Pair:', count, 'Elapsed Time:', time.time()-t0)


Band: 0 Pair: 100000 Elapsed Time: 34.4864296913147
Band: 0 Pair: 200000 Elapsed Time: 43.24747014045715
Band: 1 Pair: 100000 Elapsed Time: 79.73155903816223
Band: 1 Pair: 200000 Elapsed Time: 88.69052362442017
Band: 1 Pair: 300000 Elapsed Time: 92.0345208644867
Band: 2 Pair: 100000 Elapsed Time: 131.98002648353577
Band: 2 Pair: 200000 Elapsed Time: 140.1440007686615
Band: 3 Pair: 100000 Elapsed Time: 211.63511395454407
Band: 4 Pair: 100000 Elapsed Time: 258.0588264465332
Band: 4 Pair: 200000 Elapsed Time: 266.9956395626068
Band: 4 Pair: 300000 Elapsed Time: 270.5084662437439
Band: 5 Pair: 100000 Elapsed Time: 358.981972694397
Band: 6 Pair: 100000 Elapsed Time: 647.5936095714569
Band: 7 Pair: 100000 Elapsed Time: 801.5435185432434
Band: 7 Pair: 200000 Elapsed Time: 830.5925507545471
Band: 8 Pair: 100000 Elapsed Time: 1025.246957540512
Band: 8 Pair: 200000 Elapsed Time: 1066.0816280841827
Band: 8 Pair: 300000 Elapsed Time: 1079.005847454071
Band: 9 Pair: 100000 Elapsed Time: 1344.654632

This step goes through each hash value for each bands and produces the each combination of candidates pairs based on those binning. The combinations are then added to a dictionary to remove all duplicate candidate pairs.

In [15]:
# Initialize numpy array to store all candidate pairs
cand_pair_array = np.empty([len(cp_dict)*2,2], dtype=int)

# Loop through each candidate pair in dictionary
count = 0
t0 = time.time()
for key in cp_dict:
    
    # Split string into track of candidate pair
    can_pair_values = key.split()
    can_pair_value_0 = int(can_pair_values[0])
    can_pair_value_1 = int(can_pair_values[1])
    
    # Add candidate to array
    cand_pair_array[2*count] = [can_pair_value_0, can_pair_value_1]
    
    # Add candidate pair in reverse order to easier searching later
    cand_pair_array[2*count+1] = [can_pair_value_1, can_pair_value_0]
    
    # Display status
    count = count + 1
    if (count%25000000 == 0):
        print('Candidate Pair:', count, 'Elapsed Time:', time.time()-t0) 
    

Candidate Pair: 25000000 Elapsed Time: 100.5557336807251
Candidate Pair: 50000000 Elapsed Time: 199.9537115097046
Candidate Pair: 75000000 Elapsed Time: 302.30594992637634
Candidate Pair: 100000000 Elapsed Time: 403.317414522171
Candidate Pair: 125000000 Elapsed Time: 506.94103741645813
Candidate Pair: 150000000 Elapsed Time: 611.6281332969666
Candidate Pair: 175000000 Elapsed Time: 708.7125873565674
Candidate Pair: 200000000 Elapsed Time: 817.2377650737762
Candidate Pair: 225000000 Elapsed Time: 913.2822206020355


The candidate pair dictionary is now converted to a numpy array due to the decreased size of storage. Each candidate pair is added to the array twice: once in order 0, 1 and once in order 1, 0. Later when searching for similar songs, it will be easier being able to search just the first column of the candidate pairs array instead of both columns.

In [16]:
# Write Candidate Pair array to pickle file
t0 = time.time()
with open('./data/candidate_pair_1rows_10bands.pickle', 'wb') as f:
    pickle.dump(cand_pair_array, f, protocol=pickle.HIGHEST_PROTOCOL)

# Display Status
print('Elapsed time:', time.time()-t0)

Elapsed time: 25.784220695495605
